In [2]:
import numpy as np

# These params come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
#  DIMENSIONS specified because we are using the center of each voxel.

def get_voxel_centers_as_array(num_x_pts, num_y_pts, num_z_pts, increment, file):
    total_voxels = num_x_pts * num_y_pts * num_z_pts
    natee = 0
    next_print = total_voxels / 10
    
    voxel_centers = np.empty((num_x_pts, num_y_pts, num_z_pts), dtype=int)
    with open(file, 'r') as voxel_centers_file:
        voxel_centers_file.readline()
        for line in voxel_centers_file:
            tokens = line.split(',')
            x = int(float(tokens[0]))
            y = int(float(tokens[1]))
            z = int(float(tokens[2]))
            grain_id = int(tokens[3])
            
            # We divide the voxel_centers by the increment so that we fill every index of the list
            voxel_centers[x // increment][y // increment][z // increment] = grain_id
            
            # Print progress
            natee += 1
            if natee >= next_print:
                print('Progress: ', natee / total_voxels)
                next_print += total_voxels / 10
    return voxel_centers

In [3]:
# This will give the nearest voxel center (x,y,z) based on the voxel size being the length of 1 side of the voxel

def find_nearest_voxel_center(x, y, z, voxel_size):
    # This works because / gives a float while // gives an int. So this says that it's in one boundary until the next
    voxel_x = int(((x + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_y = int(((y + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_z = int(((z + (voxel_size / 2)) // voxel_size) * voxel_size)
    return voxel_x, voxel_y, voxel_z

In [4]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.

def get_crack_front_points(input_file):
    with open(input_file, 'r') as crack_points_file:
        crack_points_file.readline()
        crack_points = []
        for line in crack_points_file:
            tokens = line.split(',')
            crack_id = tokens[0]
            theta = tokens[1]
            x = tokens[5]
            y = tokens[6]
            z = tokens[7].strip()
            dadN3Dline = tokens[3]
            if int(crack_id) > 0:
                crack_points.append((crack_id, theta, x, y, z, dadN3Dline))
    return crack_points

In [5]:
# The 4 rightmost digits of the grain_id identify it. The other digits are to show which crack front it's on and 
#  whether it's above or below the crack front

def get_actual_grain_id(long_grain_id):
    return long_grain_id % 10000

In [6]:
from math import atan, pi

# Returns true if the given point point (x,y) is in front of the crack front at the given crack_id.
# Remember that X and Y are flipped.

def is_in_front_of_crack_surface(this_crack_front, crack_id, x, y):
    # This reference point is from the 0-crack_front_growth_rates_1500ppcf_transformed.csv file for CF_id=0
    reference_x = -1.9932
    reference_y = 363.26
    
    # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
    opposite_point = abs(x - reference_x)
    adjacent_point = abs(y - reference_y)
    
    # If the line isn't exactly vertical (i.e.: does not have undefined slope)
    if adjacent_point != 0:
        theta_point = atan(opposite_point / adjacent_point)
        # Formula from the point of reference to the point being tested: x=ay+c
        a = (x - reference_x) / (y - reference_y) # Rise over Run. X and Y are switched in this data
        c = -a * y + x # Solve x=ay+c for c and stick a point in
    
    # Otherwise, the line is vertical, so set theta to pi halves and say we don't know a and c
    else:
        theta_point = pi / 2
        a = None
        c = None
    
    # If the given point (x,y) is to the right of the reference point, then we got the wrong theta and we need to flip it
    if y > reference_y:
        theta_point = pi - theta_point
    
    # Scan the crack front until we find the two crack front points nearest theta-wise to the give (x,y)
    # This could be sped up with a binary search
    crack_index1 = 0
    crack_index2 = crack_index1 + 1
    num_pts = len(this_crack_front)
    theta_crack2 = 0
    
    while (crack_index2 + 1 < num_pts and theta_crack2 < theta_point):
        temp_crack_1x = float(this_crack_front[crack_index1][2])
        temp_crack_1y = float(this_crack_front[crack_index1][3])
        temp_crack_2x = float(this_crack_front[crack_index2][2])
        temp_crack_2y = float(this_crack_front[crack_index2][3])

        # Find the opposite and adjacent of a triangle formed between reference and given (x,y)
        opposite_crack1 = abs(temp_crack_1x - reference_x)
        adjacent_crack1 = abs(temp_crack_1y - reference_y)
        opposite_crack2 = abs(temp_crack_2x - reference_x)
        adjacent_crack2 = abs(temp_crack_2y - reference_y)

        # If the line isn't exactly vertical (i.e.: does not have undefined slope)
        if adjacent_crack1 != 0:
            theta_crack1 = atan(opposite_crack1 / adjacent_crack1)
        else:
            theta_crack1 = pi / 2

        if adjacent_crack2 != 0:
            theta_crack2 = atan(opposite_crack2 / adjacent_crack2)
        else:
            theta_crack2 = pi / 2

        if temp_crack_1y > reference_y:
            theta_crack1 = pi - temp_crack_1y
        if temp_crack_2y > reference_y:
            theta_crack2 = pi - temp_crack_2y
            
        crack_index1 += 1
        crack_index2 = crack_index1 + 1
    
    crack_1x = float(this_crack_front[crack_index1][2])
    crack_1y = float(this_crack_front[crack_index1][3])
    crack_2x = float(this_crack_front[crack_index2][2])
    crack_2y = float(this_crack_front[crack_index2][3])
    
    # If the line isn't vertical, find its slope and intercept
    if crack_1y != crack_2y:
        # Formula for points on the crack front: x = by + d
        b = (crack_1x - crack_2x) / (crack_1y - crack_2y) # Rise over Run. X and Y are switched in this data
        d = -b * crack_1y + crack_1x # Solve x = by + d for d and stick a point in
    else:
        b = None
        d = None
        
    # If we found both lines
    if a is not None and b is not None:
        # Remember X and Y are flipped
        intersection_y = (d - c) / (a - b)
        intersection_x = a * intersection_y + c
    # Otherwise, we're missing a line
    else:
        # If we got the line from the reference to the point (x,y), but not the crack front line
        if a is not None:
            intersection_x = a * intersection_y + c
            intersection_y = crack_1y # We know the crack front line is vertical and y is always the same
        
        # Otherwise, if we got the crack front line but not the point line
        elif b is not None:
            intersection_x = b * intersection_y + d
            intersection_y = y # The point line is vertical, so its y is always the same
            
        # This shouldn't happen, if it does I'll figure out how to deal with it
        else:
            raise RuntimeError('Both lines are vertical...')

    distance_point = ((y - reference_y)**2 + (x - reference_x)**2)**.5
    distance_intersection = ((intersection_y - reference_y)**2 + (intersection_x - reference_x)**2)**.5
    
    return distance_point >= distance_intersection

In [7]:
from collections import deque

# Given an array of the voxel centers arr[x][y][z] = grain_id at those 'coordinates / voxel_size'. Returns the nearest 
#  voxel center that has a different grain_id associated with it. Basically, just does a breadth-first search...
# Stuff is currently commented out because the data broke the commented out code on crack 4, it may be faster though

def find_nearest_grain_boundary(voxel_centers, checked_voxels, x, y, z, grain_id, this_crack_front, crack_id):
    grain_id = get_actual_grain_id(grain_id)
    queue = deque([(x,y,z)])
    
    while len(queue) > 0:
        u = queue.popleft()
        current_x, current_y, current_z = u
        
        # voxel_centers[current_x][current_y][current_z] is the grain_id of the voxel we're checking
        current_grain_id = get_actual_grain_id(voxel_centers[current_x][current_y][current_z])
        # TODO: Added in front of check here
        if current_grain_id != grain_id and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x, current_y):
            return (current_x, current_y, current_z, current_grain_id)
        
        checked_voxels[current_x][current_y][current_z] = True
        if current_x < len(voxel_centers) - 1 and not checked_voxels[current_x + 1][current_y][current_z]: #\
#                 and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x + 1, current_y):
            queue.append((current_x + 1, current_y, current_z))
            checked_voxels[current_x + 1][current_y][current_z] = True
            
        if current_x > 0 and not checked_voxels[current_x - 1][current_y][current_z]: #\
#                 and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x - 1, current_y):
            queue.append((current_x - 1, current_y, current_z))
            checked_voxels[current_x - 1][current_y][current_z] = True
            
        if current_y < len(voxel_centers[0]) - 1 and not checked_voxels[current_x][current_y + 1][current_z]: #\
#                 and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x, current_y + 1):
            queue.append((current_x, current_y + 1, current_z))
            checked_voxels[current_x][current_y + 1][current_z] = True
            
        if current_y > 0 and not checked_voxels[current_x][current_y - 1][current_z]: #\
#                 and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x, current_y - 1):
            queue.append((current_x, current_y - 1, current_z))
            checked_voxels[current_x][current_y - 1][current_z] = True
            
        if current_z < len(voxel_centers[0][0]) - 1 and not checked_voxels[current_x][current_y][current_z + 1]: #\
#                 and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x, current_y):
            queue.append((current_x, current_y, current_z + 1))
            checked_voxels[current_x][current_y][current_z + 1] = True
            
        if current_z > 0 and not checked_voxels[current_x][current_y][current_z - 1]: #\
#                 and is_in_front_of_crack_surface(this_crack_front, crack_id, current_x, current_y):
            queue.append((current_x, current_y, current_z - 1))
            checked_voxels[current_x][current_y][current_z - 1] = True
    
    raise RuntimeError('Grain boundary not found')

In [8]:
# Get the index in voxel_centers from x,y,z

def get_voxel_centers_index(x, y, z, voxel_size):
    return x // voxel_size, y // voxel_size, z // voxel_size

In [9]:
# Get the x,y,z from voxel_centers indices

def get_xyz_from_voxel_centers_index(xi, yi, zi, voxel_size):
    return xi * voxel_size, yi * voxel_size, zi * voxel_size

In [15]:
x=0
y=332
crack_id=1
# print(this_crack_front)
is_in_front_of_crack_surface(this_crack_front, crack_id, x, y)

False

In [187]:
# Time: ~ 3 minutes
# These params come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
#  DIMENSIONS specified because we are using the center of each voxel.

voxel_centers = get_voxel_centers_as_array(201, 376, 301, 2, '../data/1-voxel-centers.csv')
print('done')

Progress:  0.10000001758367279
Progress:  0.20000003516734557
Progress:  0.3000000087918364
Progress:  0.4000000263755092
Progress:  0.5
Progress:  0.6000000175836728
Progress:  0.7000000351673455
Progress:  0.8000000087918364
Progress:  0.9000000263755091
done


In [13]:
# Gets the crack front points from file

crack_front_points = get_crack_front_points('../../data/0-crack_front_growth_rates_1500ppcf_transformed.csv')

In [14]:
# Save the nearest grain boundaries for the points on each crack front

# Setup
voxel_size = 2
crack_id_test = '1' # Use this to do each crack front iteratively... 1 - 8 inclusive
previously_looked_up = {}
next_print = 3.15 / 100

# Set up crack_front points so we can check whether points are in front or behind later
this_crack_front = []
for crack_front_point in crack_front_points:
    this_crack_id = int(crack_front_point[0])
    if this_crack_id == int(crack_id_test):
        this_crack_front.append(crack_front_point)

# with open('../data/2-nearest_grain_boundary.csv', 'a') as file:
#     # Only write the header the first time
#     if crack_id_test == '1':
#         file.write('Crack ID, Theta, X, Y, Z, dadN3Dline, Grain ID, Nearest Grain Boundary X, Nearest Grain Boundary Y, Nearest Grain Boundary Z, Nearest Grain Boundary ID\n')
#     for crack_front_point in crack_front_points:
#         crack_id, theta, x, y, z, dadN3Dline = crack_front_point
#         if crack_id_test == crack_id:
#             x = float(x)
#             y = float(y)
#             z = float(z)

#             # Get the nearest voxel_center for the x,y,z of the crack_front
#             nearest_voxel_xyz = find_nearest_voxel_center(x, y, z, voxel_size)
#             nearest_voxel_x, nearest_voxel_y, nearest_voxel_z = nearest_voxel_xyz
            
#             # TODO: if nearest_voxel_x < 0 or nearest_voxel_y < 0 or nearest_voxel_z < 0: continue? 
#             #  Or just delete them from data so I don't have to run again this time...

#             # We need to keep track of what voxels we search
#             checked_false = np.full(voxel_centers.shape, False, np.bool)

#             # Get the grain_id of the voxel we're looking at
#             voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z = \
#                 get_voxel_centers_index(nearest_voxel_x, nearest_voxel_y, nearest_voxel_z, voxel_size)
#             nearest_grain_id = get_actual_grain_id(voxel_centers[voxel_centers_index_x][voxel_centers_index_y][voxel_centers_index_z])
            
#             # Find the x,y,z,grain_id of the nearest grain boundary
#             if nearest_voxel_xyz in previously_looked_up:
#                 grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = previously_looked_up[nearest_voxel_xyz]
#             else:
#                 grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = find_nearest_grain_boundary(voxel_centers,\
#                   checked_false, voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z, nearest_grain_id, this_crack_front, int(crack_id))
#                 previously_looked_up[nearest_voxel_xyz] = (grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id)

#             # Got back to x,y,z from indices
#             grain_boundary_x, grain_boundary_y, grain_boundary_z = get_xyz_from_voxel_centers_index(grain_boundary_index_x, \
#                                                             grain_boundary_index_y, grain_boundary_index_z, voxel_size)

#             temp = str(crack_id)+','+str(theta)+','+str(x)+','+str(y)+','+str(z)+','+str(dadN3Dline)+','+str(nearest_grain_id)+','+\
#                 str(grain_boundary_x)+','+str(grain_boundary_y)+','+str(grain_boundary_z)+','+str(grain_boundary_id)+'\n'
#             file.write(temp)

#             if float(theta) > next_print:
#                 print('Progress: ', float(theta) / 3.15)
#                 next_print += 3.15 / 100

# print('done')

In [18]:

x=-0.93115
y=332.85
z=306.54
# x=-10.547
# y=608.22
# z=358.02
voxel_size=2
find_nearest_voxel_center(x, y, z, voxel_size)

(0, 332, 306)